In [1]:
from grpc.beta import implementations
import numpy as np
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input
import numpy as np
from keras.applications.resnet50 import preprocess_input, decode_predictions
import pprint

Using TensorFlow backend.


In [2]:
#param : image from local file
#return : 1*224*224*3 matrix
def load_preprocess_img(img_path, target_size=(224, 224)):
    img = load_img(img_path, target_size=target_size)
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [15]:
#examples [cat,dog,flower,sunglass,stanforddog,sunglass]
img = load_preprocess_img('flower.jpg')
img.shape

(1, 224, 224, 3)

In [16]:
#para : img matrix
#return : json including topk classification
def predict_image_classification(img,k):
    request = predict_pb2.PredictRequest()
    request.model_spec.name = 'resnet50'
    request.model_spec.signature_name = 'predict'
    request.inputs['images'].CopyFrom(tf.contrib.util.make_tensor_proto(img))


    channel = implementations.insecure_channel("localhost", 9000)
    stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

    result = stub.Predict(request, 5)  # 10 secs timeout

    to_decode = np.expand_dims(result.outputs['scores'].float_val, axis=0)
    decoded = decode_predictions(to_decode, k)
    return decoded

In [17]:
decoded = predict_image_classification(img,5)
print(decoded)

[[(u'n11939491', u'daisy', 0.19868582487106323), (u'n02219486', u'ant', 0.16072338819503784), (u'n02264363', u'lacewing', 0.058310471475124359), (u'n02206856', u'bee', 0.049888506531715393), (u'n02280649', u'cabbage_butterfly', 0.027896983548998833)]]


In [18]:
pprint.pprint(decoded)

[[(u'n11939491', u'daisy', 0.19868582487106323),
  (u'n02219486', u'ant', 0.16072338819503784),
  (u'n02264363', u'lacewing', 0.058310471475124359),
  (u'n02206856', u'bee', 0.049888506531715393),
  (u'n02280649', u'cabbage_butterfly', 0.027896983548998833)]]
